In [8]:
import numpy as np
import pandas as pd
import time
import multiprocessing as mp
print('Number of Core :',mp.cpu_count())

class VehicleRoutingArrangement:
    def __init__(self, node_names, distances_matrix, tasks):
        self.full_node_names = node_names
        self.node_names = node_names['name'].values
        self.distances = distances_matrix
        self.tasks = tasks
        
    def checkRemainingTasks(self, job_done):
        remaining_tasks = self.tasks.copy()
        if len(remaining_tasks) > 0:
            active_path = tasks['path'].values
            job_done = job_done[np.isin(job_done, active_path)]
            
            job_unique, job_counts = np.unique(job_done, return_counts=True)
            job_done_df = pd.DataFrame({'path': job_unique, 'Count': job_counts})
            
            merged_df = remaining_tasks.merge(job_done_df, on='path',how='outer')
            merged_df.loc[merged_df['Count_y'].isnull(), 'Count_y'] = 0
            merged_df['Count'] = merged_df['Count_x'] - merged_df['Count_y']
            output = merged_df[merged_df['Count'] > 0][['Start','End','path','Count']]        
            return output
        else:
            return remaining_tasks
    
    def startOrEnd(self, current_position,rm_task):
        start_set = rm_task['Start'].unique()
        if current_position in start_set:
            return rm_task[rm_task['Start'] == current_position]['End'].values
        else:
            return start_set
        
    def nextJobsMatrix(self, job_done, next_nodes):
        current_position = job_done[-1].split('-')[-1]
        next_jobs = current_position + '-' + next_nodes
        
        if len(job_done[0]) > 1:
            ini_mat = np.tile(job_done,(len(next_jobs),1))
            output = np.hstack((ini_mat, np.array([next_jobs]).T ))
        else:
            output = np.array([next_jobs]).T
        return output
    
    def jobMatrix(self, each_row):
        remaining_jobs = self.checkRemainingTasks(each_row)
        if len(remaining_jobs) == 0:
            job_matrix = np.append(each_row,'0')
        else:
            current_position = each_row[-1].split('-')[-1]
            next_nodes = self.startOrEnd(current_position, remaining_jobs)
            job_matrix = self.nextJobsMatrix(each_row, next_nodes)
        return job_matrix
    
    def allPath(self, before_path):
        start_time = time.time()
            
        # Pool Multiple Processing    
        pool = mp.Pool(processes=4)
        output_list = pool.map(self.jobMatrix, before_path)
        output = np.vstack(output_list)

        last_column = output[:,-1]
        all_zero = sum(last_column == '0')
        stop_time = time.time() - start_time
        print('Output Matrix Shape :', output.shape, ', Use Time :', stop_time, 's')
        if all_zero == len(last_column):
            return output
        else:
            return self.allPath(output)
    
    def getDistance(self, startEndStr):
        tasks_arr = self.tasks['path'].values
        if startEndStr == '0':
            return 0
        else:
            start,end = startEndStr.split('-')
            i,j = np.where(self.node_names == start)[0][0],np.where(self.node_names == end)[0][0]
            return self.distances[i,j] if startEndStr in tasks_arr else -1*self.distances[i,j]
        
    def sumDistance(self, distance_arr):
        all_distances = sum(distance_arr)
        pos_distances = sum(distance_arr[distance_arr > 0])
        neg_distances = sum(distance_arr[distance_arr < 0])
        abs_distances = pos_distances + abs(neg_distances)
        return pos_distances, neg_distances, all_distances, abs_distances
        
    def allDistancePath(self, all_path):
        getDistanceAllElement = np.vectorize(self.getDistance) 
        result_array = getDistanceAllElement(all_path)
        all_distance_path = np.apply_along_axis(self.sumDistance, 1, result_array)
        return all_distance_path
    
    def getTaskName(self, task_path):
        name_arr = self.full_node_names['name'].values
        only_task = task_path[task_path != '0'];
        active_path = self.tasks['path'].values

        # แยกจุดเริ่ม-จบ 
        start_f = lambda x : x.split('-')[0]
        start_vf = np.vectorize(start_f)
        start_node = start_vf(only_task)
        end_f = lambda y : y.split('-')[1]
        end_vf = np.vectorize(end_f)
        end_node = end_vf(only_task)

        row_num = len(only_task)

        name_list = []
        distance_list = []
        for i in range(row_num):
            # name        
            start_name =  self.full_node_names[self.full_node_names['name'] == start_node[i]]['node'].values[0]
            end_name =  self.full_node_names[self.full_node_names['name'] == end_node[i]]['node'].values[0]
            name = start_name + '-' + end_name 
            name_list.append(name)

            # distance        
            start,end = start_node[i],end_node[i]
            m,n = np.where(name_arr == start)[0][0],np.where(name_arr == end)[0][0]
            before_path = only_task[0:i]
            remaining_tasks = self.checkRemainingTasks(before_path)
            current_task = task_path[i] 
            if current_task in remaining_tasks['path'].values:
                distance_list.append(self.distances[m,n])
            else:
                distance_list.append(-self.distances[m,n])

        return pd.DataFrame({'Name' : name_list, 'Distance': distance_list})
    
start_point = np.array([['S']]) # start_point
node_names = pd.read_csv('data/node_names.csv')
distances = pd.read_csv('data/distances_matrix.csv', header=None).values
tasks = pd.read_csv('data/demo_tasks.csv')[['Start','End','Count']]
tasks['path'] = tasks['Start'] +'-'+ tasks['End']

Number of Core : 4


In [9]:
vehicleObj = VehicleRoutingArrangement(node_names, distances, tasks)

In [10]:
all_path = vehicleObj.allPath(np.array([['A']]))

Output Matrix Shape : (2, 1) , Use Time : 0.06803369522094727 s
Output Matrix Shape : (5, 2) , Use Time : 0.0932469367980957 s
Output Matrix Shape : (11, 3) , Use Time : 0.11263108253479004 s
Output Matrix Shape : (20, 4) , Use Time : 0.1333599090576172 s
Output Matrix Shape : (42, 5) , Use Time : 0.17739200592041016 s
Output Matrix Shape : (68, 6) , Use Time : 0.231611967086792 s
Output Matrix Shape : (128, 7) , Use Time : 0.3375859260559082 s
Output Matrix Shape : (183, 8) , Use Time : 0.5194540023803711 s
Output Matrix Shape : (282, 9) , Use Time : 0.6833558082580566 s
Output Matrix Shape : (348, 10) , Use Time : 1.0800151824951172 s
Output Matrix Shape : (360, 11) , Use Time : 1.5604140758514404 s
Output Matrix Shape : (360, 12) , Use Time : 2.0010986328125 s
Output Matrix Shape : (360, 13) , Use Time : 1.6904010772705078 s
Output Matrix Shape : (360, 14) , Use Time : 1.4103810787200928 s


In [11]:
all_path.shape

(360, 14)

In [12]:
# np.save('output/all_path.npy',all_path)

In [14]:
np.load('../script/output/all_path.npy').shape

(360, 14)

In [10]:
# np.set_printoptions(threshold=np.nan,linewidth=200)
# all_path

In [15]:
distance_path = vehicleObj.allDistancePath(all_path)
np.unique(distance_path[:,2]) # เอาค่า unique

array([ 62.8,  66.9,  66.9,  76.6,  80.3,  81.8,  84.8,  85.9,  85.9,
        85.9,  89.6,  92.7,  92.7,  95.6,  96.4,  96.4,  96.8,  99.3,
       103. , 103.8, 103.8, 107.5, 110.1, 110.1, 110.6, 111.7, 111.7,
       114.3, 114.3, 114.3, 115.4, 115.4, 115.4, 115.4, 115.8, 119.1,
       119.1, 128. , 128. , 133.3, 137. ])

In [16]:
print(len(all_path[distance_path[:,2] == 137])) # จำนวนของค่าดีสุด
path_from_number_of_distances = all_path[distance_path[:,2] == 137] # เอาค่าดีสุด
# path_from_number_of_distances

24


In [17]:
thai_name = vehicleObj.getTaskName(path_from_number_of_distances[3])
thai_name

,Name,Distance
0,นครหลวง-ท่าเรือ,19.8
1,ท่าเรือ-โคกตูม,29.5
2,โคกตูม-ท่าเรือ,-29.5
3,ท่าเรือ-หนองแค(สัตว์บก),43.2
4,หนองแค(สัตว์บก)-ลำปลายมาศ,-282.0
5,ลำปลายมาศ-วังแดง,322.0
6,วังแดง-นครหลวง,-11.6
7,นครหลวง-โคกตูม,45.6
8,โคกตูม-นครหลวง,-45.6
9,นครหลวง-โคกตูม,45.6
